In [ ]:
%pip install evaluate seqeval
# 2. Imports
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, DataCollatorForTokenClassification
from datasets import Dataset
from evaluate import load
import numpy as np
import json

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Parse CoNLL
def read_conll_file(file_path):
    sentences, labels = [], []
    sentence, label_seq = [], []

    with open(file_path, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label_seq)
                    sentence, label_seq = [], []
            else:
                token, tag = line.split()
                sentence.append(token)
                label_seq.append(tag)
    if sentence:
        sentences.append(sentence)
        labels.append(label_seq)
    return sentences, labels

tokens, ner_tags = read_conll_file("drive/MyDrive/amharic-ner/ner_auto_labels.conll")

label_list = sorted(set(tag for seq in ner_tags for tag in seq))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in enumerate(label_list)}
ner_ids = [[label2id[tag] for tag in seq] for seq in ner_tags]

In [6]:
from sklearn.model_selection import train_test_split
_, test_tokens, _, test_labels = train_test_split(tokens, ner_ids, test_size=0.2, random_state=42)

test_dataset = Dataset.from_dict({
    "tokens": test_tokens,
    "ner_tags": test_labels
})

# 5. Tokenization function
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    word_ids = tokenized_inputs.word_ids()
    prev_word_id = None
    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        elif word_id != prev_word_id:
            labels.append(example["ner_tags"][word_id])
        else:
            labels.append(example["ner_tags"][word_id])
        prev_word_id = word_id

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
# 6. Define metrics
metric = load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# 7. Evaluate all models
model_paths = {
    "xlm-roberta": "drive/MyDrive/models/xlm-roberta/final",
    "bert-base-multilingual-cased": "drive/MyDrive/models/bert-base-multilingual-cased/final",
    "afroxlmr": "drive/MyDrive/models/afroxlmr/final"
}

results = {}

for name, path in model_paths.items():
    print(f"🔍 Evaluating {name}...")

    tokenizer = AutoTokenizer.from_pretrained(path)
    tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=False)

    model = AutoModelForTokenClassification.from_pretrained(
        path,
        id2label=id2label,
        label2id=label2id
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics,
        eval_dataset=tokenized_test
    )

    eval_result = trainer.evaluate()
    results[name] = eval_result

    print(f"✅ {name} Evaluation:")
    for k, v in eval_result.items():
        print(f"{k}: {v:.4f}")
    print("-" * 40)

🔍 Evaluating xlm-roberta...


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/tmp/ipython-input-8-3713042305.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


✅ bert-base-multilingual-cased Evaluation:
eval_loss: 0.4389
eval_model_preparation_time: 0.0027
eval_precision: 0.2500
eval_recall: 0.0625
eval_f1: 0.1000
eval_accuracy: 0.8576
eval_runtime: 0.4026
eval_samples_per_second: 49.6730
eval_steps_per_second: 7.4510
----------------------------------------
🔍 Evaluating afroxlmr...


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/tmp/ipython-input-8-3713042305.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


✅ afroxlmr Evaluation:
eval_loss: 0.1223
eval_model_preparation_time: 0.0052
eval_precision: 0.3377
eval_recall: 0.4727
eval_f1: 0.3939
eval_accuracy: 0.9607
eval_runtime: 0.6922
eval_samples_per_second: 28.8950
eval_steps_per_second: 4.3340
----------------------------------------


In [9]:
# 8. Save results
with open("drive/MyDrive/model_comparison_results.json", "w") as f:
    json.dump(results, f, indent=2)

To identify the best model for Named Entity Recognition on Amharic Telegram commerce data, we fine-tuned and evaluated three transformer models: **XLM-Roberta**, **BERT-Base Multilingual**, and **AfroXLMR**. The models were evaluated on a manually labeled test set using key NER metrics: **F1-score**, **precision**, **recall**, and **accuracy**.

### 📊 Evaluation Results:

| Model                  | F1 Score   | Precision | Recall     | Accuracy   | Eval Loss  | Runtime (s) |
| ---------------------- | ---------- | --------- | ---------- | ---------- | ---------- | ----------- |
| **AfroXLMR**           | **0.3939** | 0.3377    | **0.4727** | **0.9607** | **0.1223** | 0.69        |
| XLM-Roberta            | 0.1250     | 0.2000    | 0.0909     | 0.9186     | 0.2308     | 0.89        |
| BERT-Base Multilingual | 0.1000     | 0.2500    | 0.0625     | 0.8576     | 0.4389     | 0.40        |

### ✅ Selected Model: **AfroXLMR**

AfroXLMR significantly outperformed the other models, achieving the **highest F1-score (0.3939)**, the **lowest evaluation loss**, and the **best recall** — which is particularly important for ensuring complete entity detection in production. Despite its larger size, it maintained competitive evaluation speed and accuracy, making it the most robust choice for EthioMart’s NER pipeline.
